In [1]:
import torch
import numpy as np
from likelihood import *
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from data_generator import *
from formate_matrix_toMLData import * 
import os
from sklearn.model_selection import train_test_split
import Attention_model 
from model_scripts.proto_type.prototype_03.DeepSets_varSet_diagnal import *
# from model_scripts.diagonal_inverse_1to100.DeepSets_varSet_diagnal import DeepSets_varSets_forDiagnel, varSets_Datasets, collate_fn
from sklearn.preprocessing import StandardScaler
from concurrent.futures import ProcessPoolExecutor
np.set_printoptions(suppress=True)

In [2]:
#データセットように整形
states = []
del_t = []
outputs = []
formater = formate_dataMatrix()

#データ生成
# for i in range(14355,20000):
#     DTRMG = DiagonalTransitionRateMatrixGenerator(4)
#     Q = DTRMG.generateMatrix()
#     dg = DataGenerator(Q, 50000)
#     all_matrix = dg.generate_matrix()
#     dg.generate_dataFile(all_matrix,f"data_{str(i)}_", "data/diagonal_setSize50000")
#     tm = matrix_trimer(all_matrix)
#     trm = tm.trim_transitionRateMatrix()
#     data = tm.trim_data()
#     output_vec = np.array(formater.GetOutputVector_byUpperTriangle(trm))
#     # states に追加（各要素は (2, num_samples_i) の形状）
#     state = np.stack([data[:,0], data[:,1]], axis=0)# shape: (2, num_samples_i)
#     states.append(state)
#     # del_t に追加（各要素は (num_samples_i,) の形状）
#     del_t.append(data[:, 2])
#     # outputs に追加（各要素は (6,) の形状）
#     outputs.append(output_vec)
#     print(i)


In [3]:
states = []
del_t = []
outputs = []


In [ ]:

def process_file(file_path):
    #ファイルの読み込み
    with open(file_path,encoding="utf-8") as f:
        all_matrix = np.loadtxt(f, delimiter= ",")
    tm = matrix_trimer(all_matrix)
    trm = tm.trim_transitionRateMatrix()
    data = tm.trim_data()
    output_vec = np.array(formater.GetOutputVector_byDiagonal(trm))
    # data: shape (num_samples_i, 3)
    # data[:, 0]: pre 状態
    # data[:, 1]: post 状態
    # data[:, 2]: delta_t
    
    
    
    
    # states に追加（各要素は (2, num_samples_i) の形状）
    state = np.stack([data[:,0], data[:,1]], axis=0)# shape: (2, num_samples_i)
    states.append(state)
    
    # del_t に追加（各要素は (num_samples_i,) の形状）
    del_t.append(data[:, 2])
    
    # outputs に追加（各要素は (3,) の形状）
    outputs.append(output_vec)

: 

In [ ]:
# formater.process_all_files_in_directory("/Users/yamashitanatsuki/Desktop/data/diagonal_50000_val_ratio", process_file)
formater.process_all_files_in_directory("/Users/yamashitanatsuki/Desktop/data/diagonal_50000_val_ratio04_uniform_life",process_file,end=50000)
# データの分割
# 1. トレーニング＋検証 と テスト に分割
train_states, val_states, train_del_t, val_del_t, train_outputs, val_outputs = train_test_split(
    states, del_t, outputs, test_size=0.1, random_state=42
)

# 2. トレーニング と 検証 に分割
# train_states, val_states, train_del_t, val_del_t, train_outputs, val_outputs = train_test_split(
#     train_val_states, train_val_del_t, train_val_outputs, test_size=0.25, random_state=42
# )

# データサイズの確認
print(f"Total data size: {len(states)}")
print(f"Training data size: {len(train_states)}")
print(f"Validation data size: {len(val_states)}")
# print(f"Test data size: {len(test_states)}")



# データセットとデータローダーの作成
batch_size = 128


train_dataset = varSets_Datasets(train_states, train_del_t, train_outputs)
val_dataset = varSets_Datasets(val_states, val_del_t, val_outputs)
# test_dataset = varSets_Datasets(test_states, test_del_t, test_outputs)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:
# for a, b, c, d in train_loader:
#     print(a)

In [ ]:
device = torch.device("cpu")
model = Attention_model.DeepSets_AttentionAggregate(num_categories=4, embedding_dim=2).to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)
class CustomLoss(nn.Module):
    def forward(self, outputs, targets):
         # 安全のため、小さい値を追加してゼロ割りを防止
        epsilon = 1e-12
        y_pred_inverse = 1.0 / (outputs + epsilon)
        y_true_inverse = 1.0 / (targets + epsilon)
        
        loss = torch.abs(y_pred_inverse-y_true_inverse).mean()
        
        # 平均損失を計算
        return loss
criterion = CustomLoss()

class KLloss(nn.Module):
    def forward(self, outputs,targets):
        kl_div = torch.log(targets / outputs) + (outputs / targets)
        return kl_div.sum()
            

In [ ]:
# 早期終了の設定
num_epochs = 10  # 最大エポック数
patience = 10     # 検証損失が改善しない許容エポック
best_val_loss = float("inf")
patience_counter = 0
# loss_func = KLloss()
best_model_path = 'Attention_model.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for states_batch, del_t_batch, targets_batch, lengths in train_loader:
        optimizer.zero_grad()
        outs = model(states_batch,del_t_batch, lengths)
        loss = criterion(outs, targets_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * states_batch.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.6f}")

    # 検証ループ
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for val_states_batch, val_del_t_batch, val_targets_batch, val_lengths in val_loader:
            val_outs = model(val_states_batch, val_del_t_batch, val_lengths)
            val_loss = criterion(val_outs, val_targets_batch)
            val_running_loss += val_loss.item() 
    val_epoch_loss = val_running_loss / len(val_dataset)
    print(f"Validation Loss: {val_epoch_loss:.6f}")

    # 検証損失の改善を確認
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        patience_counter = 0
        # 最良モデルの保存
        # --- 保存 ---
        torch.save(model.state_dict(), best_model_path)

        print("Validation loss improved. Model saved.")
    else:
        patience_counter += 1
        print(f"No improvement in validation loss for {patience_counter} epochs.")

    # 早期終了の判定
    if patience_counter >= patience:
        print("Early stopping triggered.")
        break

Epoch 1/10, Training Loss: 30.481714
Validation Loss: 0.211278
Validation loss improved. Model saved.
Epoch 2/10, Training Loss: 27.299786
Validation Loss: 0.193623
Validation loss improved. Model saved.
Epoch 3/10, Training Loss: 25.221632
Validation Loss: 0.169558
Validation loss improved. Model saved.
Epoch 4/10, Training Loss: 23.382277
Validation Loss: 0.139821
Validation loss improved. Model saved.
Epoch 5/10, Training Loss: 20.177115
Validation Loss: 0.128122
Validation loss improved. Model saved.
Epoch 6/10, Training Loss: 18.888176
Validation Loss: 0.127082
Validation loss improved. Model saved.
Epoch 7/10, Training Loss: 18.036740
Validation Loss: 0.123679
Validation loss improved. Model saved.
Epoch 8/10, Training Loss: 16.587644
Validation Loss: 0.083750
Validation loss improved. Model saved.
Epoch 9/10, Training Loss: 14.567283
Validation Loss: 0.073733
Validation loss improved. Model saved.
Epoch 10/10, Training Loss: 13.490118
Validation Loss: 0.092267
No improvement in 

In [ ]:
# model.eval()
# test_running_loss = 0
# with torch.no_grad():
#     for test_states_batch, test_del_t_batch, test_target_batch, test_lengths in test_loader:
#         test_outs = model(test_states_batch, test_del_t_batch,test_lengths)
#         test_loss = criterion(test_outs, test_target_batch)
#         test_running_loss += test_loss.item() * test_target_batch.size(1)
        
# test_epoch_loss = test_running_loss/len(test_dataset)
# print(f"test Loss: {val_epoch_loss:.6f}")
        
        

In [ ]:
# import model_scripts.diagonal_var_initial_ratio_02.DeepSets_varSet_diagnal  as m
# # 早期終了の設定
# num_epochs = 1000  # 最大エポック数
# patience = 10     # 検証損失が改善しない許容エポック数
# best_val_loss = float("inf")
# patience_counter = 0
# model = m.DeepSets_varSets_forDiagnel(3,2)
# best_model_path = 'model_scripts/diagonal_var_initial_ratio_02/diagonal_var_ratio_.pth'
# loss_func = CustomLoss()
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for states_batch, del_t_batch, targets_batch, lengths in train_loader:
#         optimizer.zero_grad()
#         outs = model(states_batch,del_t_batch, lengths)
#         loss = criterion(outs, targets_batch)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item() * states_batch.size(0)
#     epoch_loss = running_loss / len(train_dataset)
#     print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.6f}")

#     # 検証ループ
#     model.eval()
#     val_running_loss = 0.0
#     with torch.no_grad():
#         for val_states_batch, val_del_t_batch, val_targets_batch, val_lengths in val_loader:
#             val_outs = model(val_states_batch, val_del_t_batch, val_lengths)
#             val_loss = criterion(val_outs, val_targets_batch)
#             val_running_loss += val_loss.item() 
#     val_epoch_loss = val_running_loss / len(val_dataset)
#     print(f"Validation Loss: {val_epoch_loss:.6f}")

#     # 検証損失の改善を確認
#     if val_epoch_loss < best_val_loss:
#         best_val_loss = val_epoch_loss
#         patience_counter = 0
#         # 最良モデルの保存
#         torch.save(model, best_model_path)
#         print("Validation loss improved. Model saved.")
#     else:
#         patience_counter += 1
#         print(f"No improvement in validation loss for {patience_counter} epochs.")

#     # 早期終了の判定
#     if patience_counter >= patience:
#         print("Early stopping triggered.")
#         break